<a href="https://colab.research.google.com/github/Aditya-Walia1/Demo/blob/main/Optimised_bert_based_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install transformers datasets scikit-learn

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from transformers import EarlyStoppingCallback

# Load your dataset
file_path = '/content/Constraint_Train.csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure
print(df.head())

# Inspect the column names to identify the text and label columns
print(df.columns)

# Update these variables with the actual column names in your dataset
text_column = 'tweet'  # Replace with the actual text column name
label_column = 'label'  # Replace with the actual label column name

# Encode the labels as integers
label_encoder = LabelEncoder()
df[label_column] = label_encoder.fit_transform(df[label_column])

# Verify the encoding (optional)
print("Encoded labels:", label_encoder.classes_)

# Preprocess the dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split the dataset into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[text_column].values,
    df[label_column].values,
    test_size=0.2,
    random_state=42
)

# Define max token length
MAX_LEN = 128

# Create train and validation datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, MAX_LEN)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(df[label_column].unique())
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Increase epochs to 5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,  # Load best model at the end of training
    metric_for_best_model="f1",  # Set the metric for best model
)

# Implement Early Stopping
callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]

# Initialize Trainer with the updated metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=callbacks
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# Predictions for validation data
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)

# Calculate and print detailed classification metrics
accuracy = accuracy_score(val_labels, preds)
conf_matrix = confusion_matrix(val_labels, preds)
class_report = classification_report(val_labels, preds, target_names=label_encoder.classes_)
f1 = f1_score(val_labels, preds, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Weighted F1 Score: {f1}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


   id                                              tweet label
0   1  The CDC currently reports 99031 deaths. In gen...  real
1   2  States reported 1121 deaths a small rise from ...  real
2   3  Politically Correct Woman (Almost) Uses Pandem...  fake
3   4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4   5  Populous states can generate large case counts...  real
Index(['id', 'tweet', 'label'], dtype='object')
Encoded labels: ['fake' 'real']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.204200,0.139930,0.960280,0.960316
2,0.138800,0.168597,0.957165,0.957218
3,0.002000,0.116412,0.975857,0.975829
4,0.000400,0.116834,0.978972,0.978980
5,0.000100,0.135762,0.978193,0.978173


Evaluation Results: {'eval_loss': 0.11683408915996552, 'eval_accuracy': 0.9789719626168224, 'eval_f1': 0.9789795850403314, 'eval_runtime': 9.7389, 'eval_samples_per_second': 131.843, 'eval_steps_per_second': 8.317, 'epoch': 5.0}
Accuracy: 0.9789719626168224
Weighted F1 Score: 0.9789795850403314
Confusion Matrix:
 [[586  10]
 [ 17 671]]
Classification Report:
               precision    recall  f1-score   support

        fake       0.97      0.98      0.98       596
        real       0.99      0.98      0.98       688

    accuracy                           0.98      1284
   macro avg       0.98      0.98      0.98      1284
weighted avg       0.98      0.98      0.98      1284

